*Klausurtagung 2019*

# IMIP-Solvers and Julia-OPT

Nikolai Stein, Toni Greif<br>
Lehrstuhl für Wirtschaftsinformatik und Informationsmanagement

*Rothenburg ob der Tauber, August 7-9, 2019*

# Mathematical Optimization
- Specialized solvers for linear, mixed integer, quadratic, etc. problems
- Need to be able to write code that is a natural and maintainable translation of the mathematics of our problem
- Preferably loosley coupled to the specific solution method and data

# What is JuliaOpt?
- A collection of high-quality optimization related packages
- Algebraic modelling language that lets you express optimization problems naturally and quickly (JuMP)
- Ligh-weight wrappers around C interfaces of powerfult commercial and open-source solvers:
    - Gurobi
    - COIN-OR
    - GLPK
    - CPLEX
    - ...

# Why should we use it?
+ Very user-friendly
+ Speed similar to special purpose commercial modelling language like AMPL
+ Solver independent code: same code will run for both commercial and open-source solvers
+ Very easy to implement solver callback and problem modification

# JuMP: Algebraic modeling
- JuMP allows us to translate mathematical statements of optimization problems into code
- It efficiently generates the data structures solvers require as input
- Much easier to reason about than hand-coding problem structure

# JuMP Philosophy
- Make modeling quick and painless for practitioners
- Allow experts to easilty use integrated advanced features (usually available through low-level solver interfaces)
- Make it as fast as possible

# Install Julia

To use Julia in local notebooks you have to:
- Install anaconda ([Anaconda Downloads](https://www.anaconda.com/distribution/))
- Download Julia for your OS ([Julia Downloads](https://julialang.org/downloads/))
- Add the Julia Kernel to Jupyter ([Tutorial](https://datatofish.com/add-julia-to-jupyter/))

Or you can just use Google Colab. Julia is not installed, you will need to run the installation notebook at:

 [docs/colab/InstallJuliaXLA.ipynb](https://colab.research.google.com/github/JuliaTPU/XLA.jl/blob/master/docs/colab/InstallJuliaXLA.ipynb)
 
first. Afterwards, any JuliaTPU notebook should work if opened without resetting the runtime in between.

# Install JuMP and Solvers
For an exhaustive list of available solvers see [solvers](http://www.juliaopt.org/JuMP.jl/v0.14/installation.html#getting-solvers)

In [1]:
using Pkg
Pkg.add("JuMP")

Pkg.add("GLPK")
Pkg.add("Gurobi")

  Updating registry at `C:\Users\Toni Greif\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `C:\Users\Toni Greif\.julia\environments\v1.1\Project.toml`
 [no changes]
  Updating `C:\Users\Toni Greif\.julia\environments\v1.1\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Toni Greif\.julia\environments\v1.1\Project.toml`
 [no changes]
  Updating `C:\Users\Toni Greif\.julia\environments\v1.1\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Toni Greif\.julia\environments\v1.1\Project.toml`
 [no changes]
  Updating `C:\Users\Toni Greif\.julia\environments\v1.1\Manifest.toml`
 [no changes]


# A simple example
First, we solve a very simple example to get familiar with JuMP:

$\max 30x + 20y$

subject to:

$x \leq 10$

$y \leq 6$

$2x + 4y \leq 32$

$x, y \geq 0$

Load JuMP and the GLPK solver:

In [2]:
using JuMP
using GLPK

Construct the model:

In [3]:
myFirstModel = Model(with_optimizer(GLPK.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

Define variables:

In [4]:
@variable(myFirstModel, 0 <= x <= 10)
@variable(myFirstModel, 0 <= y <= 6);

Set objective:

In [5]:
@objective(myFirstModel, Max, 30*x+20*y)

30 x + 20 y

Add constraints:

In [6]:
@constraint(myFirstModel, 2*x + 4*y <= 32)

2 x + 4 y <= 32.0

Print the model:

In [7]:
print(myFirstModel)

Max 30 x + 20 y
Subject to
 x >= 0.0
 y >= 0.0
 x <= 10.0
 y <= 6.0
 2 x + 4 y <= 32.0


Solve the model:

In [8]:
optimize!(myFirstModel)

Print the results:

In [9]:
println("Model status: ", termination_status(myFirstModel))
println("Objective value: ", objective_value(myFirstModel))
println("x = ", value(x)) #
println("y = ", value(y))

Model status: OPTIMAL
Objective value: 360.0
x = 10.0
y = 3.0


# An advanced Example - Bin Packing Problem

Given items of different weights and bins each of a different weight capacity and fixed costs, assign each item to a bin such that the total costs are minimized.

In [10]:
weight = [0.2; 0.5; 0.4; 0.7; 0.1; 0.3; 0.8]

capacity = [1; 2; 1; 3]

costs_fix = [2; 5; 2; 10]

items = 1:size(weight,1)
bins = 1:size(capacity,1);

Load Linear Algebra functions (e.g. for the sum function):

In [11]:
using LinearAlgebra

In this example we want to create our model solver independantly.

In [12]:
bp_mod = Model()

# binary varible y_j = 1 if bin i is used
@variable(bp_mod, y[j=bins], Bin)
# binary varible x_ij = 1 if item i is assigned to bin j
@variable(bp_mod, x[ i=items, j=bins], Bin)

# minimize costs
@objective(bp_mod, Min, dot(costs_fix, y))

# subject to
for j in bins 
    @constraint(bp_mod, sum(x[i,j] * weight[i] for i in items) <= capacity[j] * y[j])   
end

for i in items 
    @constraint(bp_mod, sum(x[i,j] for j in bins) == 1)

end

We are now ready to solve the problem. We use the GLPK solver we installed before.

In [13]:
optimize!(bp_mod, with_optimizer(GLPK.Optimizer));

In [14]:
println("Model status: ", termination_status(bp_mod))
println("Objective value: ", objective_value(bp_mod))
println("x = ", value.(x)) #
println("y = ", value.(y))

Model status: OPTIMAL
Objective value: 7.0
x = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:7
    Dimension 2, 1:4
And data, a 7×4 Array{Float64,2}:
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
y = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Array{Float64,1}:
 0.0
 1.0
 1.0
 0.0


We extend our model with variable costs. Therefore, we copy the basic model.

In [15]:
# each bin j has variable costs for each item
costs_var = [2; 3; 3; 5];

In [16]:
extended_mod  = copy(bp_mod)
x_new = copy(x, extended_mod)
y_new = copy(y, extended_mod);

For the better result presentation we introduce additional integer variables which count the number of items in each bin:

In [17]:
@variable(extended_mod, z[j=bins], Int)

for j in bins 
    @constraint(extended_mod, sum(x_new[i,j] for i in items) == z[j])   
end;

Set the new objective:

In [18]:
@objective(extended_mod, Min, dot(costs_fix, y_new) + sum(z[j]*costs_var[j] for j=bins));

In [19]:
print(extended_mod)

Min 2 y[1] + 5 y[2] + 2 y[3] + 10 y[4] + 2 z[1] + 3 z[2] + 3 z[3] + 5 z[4]
Subject to
 y[1] binary
 y[2] binary
 y[3] binary
 y[4] binary
 x[1,1] binary
 x[1,2] binary
 x[1,3] binary
 x[1,4] binary
 x[2,1] binary
 x[2,2] binary
 x[2,3] binary
 x[2,4] binary
 x[3,1] binary
 x[3,2] binary
 x[3,3] binary
 x[3,4] binary
 x[4,1] binary
 x[4,2] binary
 x[4,3] binary
 x[4,4] binary
 x[5,1] binary
 x[5,2] binary
 x[5,3] binary
 x[5,4] binary
 x[6,1] binary
 x[6,2] binary
 x[6,3] binary
 x[6,4] binary
 x[7,1] binary
 x[7,2] binary
 x[7,3] binary
 x[7,4] binary
 z[1] integer
 z[2] integer
 z[3] integer
 z[4] integer
 x[1,1] + x[1,2] + x[1,3] + x[1,4] == 1.0
 x[2,1] + x[2,2] + x[2,3] + x[2,4] == 1.0
 x[3,1] + x[3,2] + x[3,3] + x[3,4] == 1.0
 x[4,1] + x[4,2] + x[4,3] + x[4,4] == 1.0
 x[5,1] + x[5,2] + x[5,3] + x[5,4] == 1.0
 x[6,1] + x[6,2] + x[6,3] + x[6,4] == 1.0
 x[7,1] + x[7,2] + x[7,3] + x[7,4] == 1.0
 x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] + x[6,1] + x[7,1] - z[1] == 0.0
 x[1,2] + x[2,2]

This time we want to use the gurobi solver:

In [20]:
using Gurobi

┌ Info: Recompiling stale cache file C:\Users\Toni Greif\.julia\compiled\v1.1\Gurobi\do9v6.ji for Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1184


Maybe you can have issues to load gurobi if the environment is not configured:

In [21]:
Pkg.build("Gurobi")
using Gurobi

  Building Gurobi → `C:\Users\Toni Greif\.julia\packages\Gurobi\dlJep\deps\build.log`


We are now ready to solve the extended problem:

In [22]:
optimize!(extended_mod, with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 15 rows, 36 columns and 92 nonzeros
Variable types: 0 continuous, 36 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 28.0000000
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolved: 11 rows, 32 columns, 60 nonzeros
Variable types: 0 continuous, 32 integer (32 binary)

Root relaxation: objective 2.350000e+01, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   23.50000    0    3   28.00000   23.50000  16.1%     -    0s
H    0     0                      26.0000000   23.50000  9.62%     -    0s
*    0     0               0      24.0000000   24.00000  0.0

In [23]:
println("Model status: ", termination_status(extended_mod))
println("Objective value: ", objective_value(extended_mod))
println("x = ", value.(x_new))
println("y = ", value.(y_new))
println("z = ", value.(z))

Model status: OPTIMAL
Objective value: 24.0
x = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:7
    Dimension 2, 1:4
And data, a 7×4 Array{Float64,2}:
  1.0  -0.0  -0.0  -0.0
 -0.0   1.0   0.0  -0.0
  1.0   0.0  -0.0  -0.0
 -0.0   1.0  -0.0  -0.0
  1.0  -0.0  -0.0  -0.0
  1.0  -0.0  -0.0  -0.0
 -0.0   1.0  -0.0   0.0
y = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Array{Float64,1}:
  1.0
  1.0
  0.0
 -0.0
z = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Array{Float64,1}:
 4.0
 3.0
 0.0
 0.0


# Credits
Parts of this presenation are adapted from:
- https://github.com/JuliaCon/presentations/blob/master/JuliaOpt/JuliaOpt.pdf
- http://www.juliaopt.org/notebooks/Shuvomoy%20-%20Getting%20started%20with%20JuMP.html